# AOC 2024

Welcome to the Advent of Code 2024!

## Basic configuration



In [ ]:
! pip install -U pip advent-of-code-data numpy pandas networkx matplotlib scipy

In [ ]:
import os

os.environ['AOC_SESSION'] = open('session.txt').read().strip()

In [ ]:
from aocd.models import Puzzle
from collections import Counter, defaultdict, deque
from helpers.functions import create_grid
from itertools import chain, product, combinations, permutations, zip_longest
from numpy.typing import ArrayLike
from pprint import pprint
from queue import PriorityQueue
from scipy import sparse
from statistics import median
from typing import Callable, Dict, Iterator, Mapping, Tuple
from IPython.display import display, clear_output
import copy
import functools
import math
import operator
import networkx as nx
import numpy as np
import pandas as pd
import sys
import re
import time

## Day 25
https://adventofcode.com/2024/day/25

In [ ]:
puzzle = Puzzle(year=2024, day=25)
input_data = """#####
.####
.####
.####
.#.#.
.#...
.....

#####
##.##
.#.##
...##
...#.
...#.
.....

.....
#....
#....
#...#
#.#.#
#.###
#####

.....
.....
#.#..
###..
###.#
###.#
#####

.....
.....
.....
#....
#.#..
#.#.#
#####"""

DEBUG = False

data = input_data if DEBUG else puzzle.input_data

### Part 1

In [ ]:
def add_lock(lines):
    lines = lines.splitlines()
    results = defaultdict(lambda: None)
    
    for nb, line in enumerate(lines[1:]):
        for col in range(5):
            if results[col] is None and line[col] == ".":
                results[col] = nb
    return list(results.values())

def add_key(lines):
    lines = lines.splitlines()
    lines = lines[::-1]
    results = defaultdict(lambda: None)
    
    for nb, line in enumerate(lines[1:]):
        for col in range(5):
            if results[col] is None and line[col] == ".":
                results[col] = nb
    return list(results.values())

In [ ]:
locks = []
keys = []

for line in data.split("\n\n"):
    if line.startswith("#####\n"):
        locks.append(add_lock(line))
    else:
        keys.append(add_key(line))


In [ ]:
total = 0

for lock in locks:
    
    for key in keys:
        if all([x[0] + x[1] <= 5 for x in zip(lock, key)]):
            total += 1
        
if DEBUG:
    print(total)
else:
    puzzle.answer_a = total

## Day 24
https://adventofcode.com/2024/day/24

In [ ]:
puzzle = Puzzle(year=2024, day=24)

input_data = """x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02"""

input_data = """x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj"""

DEBUG = False

data = puzzle.input_data.replace(" -> z07", " -> XXX")
data = data.replace(" -> nqk", " -> z07")
data = data.replace(" -> XXX", " -> nqk")

data = data.replace(" -> fgt", " -> XXX")
data = data.replace(" -> pcp", " -> fgt")
data = data.replace(" -> XXX", " -> pcp")

data = data.replace(" -> z24", " -> XXX")
data = data.replace(" -> fpq", " -> z24")
data = data.replace(" -> XXX", " -> fpq")

data = data.replace(" -> z32", " -> XXX")
data = data.replace(" -> srn", " -> z32")
data = data.replace(" -> XXX", " -> srn")

# data = input_data if DEBUG else puzzle.input_data

### Part 1

In [ ]:
@functools.lru_cache
def operation(op, lit1, lit2):
    if op == "AND":
        return lit1 & lit2
    elif op == "XOR":
        return lit1 ^ lit2
    elif op == "OR":
        return lit1 | lit2
    else:
        raise ValueError((op, lit1, lit2))

In [ ]:
inputs, rules_str = data.split("\n\n")
signals = defaultdict(lambda: None)
rules = defaultdict(None)

for line in inputs.splitlines():
    signal, value = line.split(": ")
    signals[signal] = int(value)
    
for line in rules_str.splitlines():
    m = re.match(r"([\w\d]+) (\w+) ([\w\d]+) -> ([\w\d]+)", line)
    lit1, op, lit2, res = m.groups()
    rules[res] = (op, lit1, lit2)

while rules:
    # print("S", signals)
    # print("R", rules)
    # print()

    cp_rules = copy.deepcopy(rules)
    
    for res, rule in cp_rules.items():
        if signals[res] is not None:
            continue
            
        op, lit1, lit2 = rule

        if isinstance(lit1, str) and signals[lit1] is not None:
            rules[res] = (op, signals[lit1], lit2)
            
        op, lit1, lit2 = rules[res]

        if isinstance(lit2, str) and signals[lit2] is not None:
            rules[res] = (op, lit1, signals[lit2])
            
        if isinstance(rules[res][1], int) and isinstance(rules[res][2], int):
            signals[res] = operation(*rules[res])
            del rules[res]
    
sorted_z = sorted([x for x in signals.items() if x[0].startswith("z")], key=lambda x: x[0], reverse=True)
z_dec = int("".join([str(zi[1]) for zi in sorted_z]), 2)

if DEBUG:
    print(z_dec)
else:
    puzzle.answer_a = z_dec

### Part 2

In [ ]:
inputs, rules_str = data.split("\n\n")
signals = defaultdict(lambda: None)
rules = defaultdict(None)

for line in inputs.splitlines():
    signal, value = line.split(": ")
    signals[signal] = int(value)
    
for line in rules_str.splitlines():
    m = re.match(r"([\w\d]+) (\w+) ([\w\d]+) -> ([\w\d]+)", line)
    lit1, op, lit2, res = m.groups()
    rules[res] = (op, lit1, lit2)
    
    
# Trying to match :
#     - xor_i = xi^yi, 
#     - and_i = xi&yi,
#     - tmp_and_i = tmp_or_i-1 & xor_i-1,
#     - tmp_or_i = tmp_and_i | and_i-1,
#     - z_i = tmp_or_i ^ xor_i

xors = defaultdict(str)
ands = defaultdict(str)

cp_rules = copy.deepcopy(rules)
for res, rule in cp_rules.items():
    op, lit1, lit2 = rule
    
    re_m = re.match(r"[xy](\d\d)[xy](\d\d)", f"{lit1}{lit2}")
    
    if re_m:
        idx = re_m.group(1)
        
        if op == "XOR":
            xors[int(idx)] = (res, rule)
            del rules[res]
            
        elif op == "AND":
            ands[int(idx)] = (res, rule)
            del rules[res]

In [ ]:
# Trying to match :
#     - xor_i = xi^yi, 
#     - and_i = xi&yi,
#     - tmp_and_i = tmp_or_i-1 & xor_i-1,
#     - tmp_or_i = tmp_and_i | and_i-1,
#     - z_i = tmp_or_i ^ xor_i


tmp_ands = defaultdict(str)
tmp_ors = defaultdict(str)
zs = defaultdict(str)

for i in range(2, 46):
    breaked = False
    for res, rule in rules.items():
        op, lit1, lit2 = rule
        
        if op == "AND":
            if (i == 2 and {lit1, lit2} == {ands[0][0], xors[i - 1][0]}) or (i>2 and {lit1, lit2} == {tmp_ors[i - 1][0], xors[i - 1][0]}):
                print(rule)
                tmp_ands[i] = (res, rule)
                breaked = True
                break

    if breaked:
        del rules[res]
    else:
        print("did not break")
    breaked = False
    
    for res, rule in rules.items():
        op, lit1, lit2 = rule
        
        if op == "OR" and {lit1, lit2} == {ands[i - 1][0], tmp_ands[i][0]}:
            print(rule)
            tmp_ors[i] = (res, rule)
            breaked = True
            break
    
    if breaked:
        del rules[res]
    else:
        print("did not break")
    breaked = False
    
    for res, rule in rules.items():
        op, lit1, lit2 = rule
        
        if op == "XOR" and {lit1, lit2} == {tmp_ors[i][0], xors[i][0]}:
            print(rule)
            zs[i] = (res, rule)
            breaked = True
            break
            
    if breaked:
        del rules[res]
    else:
        print("did not break")
    breaked = False

In [ ]:
puzzle.answer_b = ",".join(sorted(["pcp", "fgt", "nqk", "z07", "z24", "fpq", "srn", "z32"]))

## Day 23
https://adventofcode.com/2024/day/23

In [ ]:
puzzle = Puzzle(year=2024, day=23)
input_data = """"""

DEBUG = False
data = input_data if DEBUG else puzzle.input_data

### Part 1

In [ ]:
G = nx.Graph()

for line in data.splitlines():
    nodes = line.split("-")
    
    for n in nodes:
        G.add_node(n)
    
    G.add_edge(*nodes)

In [ ]:
total = 0
nodes_with_t = [node for node in G.nodes if node.startswith("t")]

for clique in nx.enumerate_all_cliques(G):
    if len(clique) == 3 and any([c in nodes_with_t for c in clique]):
        total += 1

puzzle.answer_a = total

### Part 2

In [ ]:
puzzle.answer_b = ",".join(sorted(clique))

## Day 22
https://adventofcode.com/2024/day/22

In [ ]:
puzzle = Puzzle(year=2024, day=22)
input_data = """1
10
100
2024"""

input_data = """1
2
3
2024"""

DEBUG = True
data = input_data if DEBUG else puzzle.input_data

### Part 1

In [ ]:
@functools.lru_cache(maxsize=10000000)
def step1(secret):
    return prune(mix(secret * 64, secret))
    
@functools.lru_cache(maxsize=10000000)
def step2(secret):
    return prune(mix(secret // 32, secret))

@functools.lru_cache(maxsize=10000000)
def step3(secret):
    return prune(mix(secret * 2048, secret))

@functools.lru_cache(maxsize=10000000)
def mix(value, secret):
    return value ^ secret

@functools.lru_cache(maxsize=10000000)
def prune(secret):
    return secret % 16777216

In [ ]:
total = 0

for secret in data.splitlines():
    secret = int(secret)
    for x in range(2000):
        secret = step3(step2(step1(secret)))
    total += secret
    
if DEBUG:
    print(total)
else:
    puzzle.answer_a = total

### Part 2

In [ ]:
sequences = []
top_scores = defaultdict(list)

for secret in data.splitlines():
    secret = int(secret)
    sequence = [secret % 10]
    
    for x in range(2000):
        secret = step3(step2(step1(secret)))
        sequence.append(secret % 10)
    
    sequences.append([x[1] - x[0] for x in zip(sequence[:-1], sequence[1:])])
    
    # print(sequences[-1])
    
    top_score_here = {}
    for i in range(0, 1997):
        if tuple(sequences[-1][i:i+4]) not in top_score_here:
            top_score_here[tuple(sequences[-1][i:i+4])] = sequence[i+4]
        
    for k, v in top_score_here.items():
        top_scores[k].append(v)
        
    # print(top_score_here)
    
total = {}
for k, v in top_scores.items():
    total[k] = sum(v)
    
sorted(total.items(), key=lambda x: x[1], reverse=True)

## Day 21
https://adventofcode.com/2024/day/21

In [ ]:
puzzle = Puzzle(year=2024, day=21)
input_data = """029A
980A
179A
456A
379A"""

DEBUG = False
data = input_data if DEBUG else puzzle.input_data

In [ ]:
G_num = nx.grid_2d_graph(4, 3) # 789, 456, 123, x0A
G_dir = nx.grid_2d_graph(2, 3) # x^A, <v>

G_num.remove_node((3, 0))
G_dir.remove_node((0, 0))

NUM = {
    "7": (0, 0),
    "8": (0, 1),
    "9": (0, 2),
    "4": (1, 0),
    "5": (1, 1),
    "6": (1, 2),
    "1": (2, 0),
    "2": (2, 1),
    "3": (2, 2),
    "0": (3, 1),
    "A": (3, 2),
}

DIR = {
    "^": (0, 1),
    "A": (0, 2),
    "<": (1, 0),
    "v": (1, 1),
    ">": (1, 2),
}

### Part 1

In [ ]:
@functools.lru_cache(maxsize=10000)
def find_all_paths_for_robot(graph, robot_source, robot_dest, display=""):
    # print("F", robot_source, robot_dest, display)
    for path in nx.all_shortest_paths(graph, robot_source, robot_dest):
        # print("P", path, display)
        yield path
    
@functools.lru_cache(maxsize=10000)
def movements_for_paths(paths):
    movements = []
    for path in paths:
        movement = []
        for before, after in zip(path[:-1], path[1:]):
            dr, dc = (after[0] - before[0], after[1] - before[1])
            
            if (dr, dc) == (1, 0):
                movement.append("v")
            elif (dr, dc) == (-1, 0):
                movement.append("^")
            elif (dr, dc) == (0, 1):
                movement.append(">")
            elif (dr, dc) == (0, -1):
                movement.append("<")
            else:
                raise ValueError((dr, dc))
        
        movements.append(movement + ["A"])
    return movements


def move_robot(symb, alphabet, initial_pos, graph, display=""):
    
    # print("D", alphabet[symb], display)
    
    paths_robot = find_all_paths_for_robot(graph, initial_pos, alphabet[symb], display=display)
    movements_next = movements_for_paths(paths_robot)
    
    
    return movements_next

@functools.lru_cache(maxsize=10000)
def find_shortest_sequence1(value):
    
    robot1_pos = (3, 2)
    robot2_pos = (0, 2)
    robot3_pos = (0, 2)
    
    final_seq = []
    r3_seq = []
    
    for char in value:
        print(char)
        
        movements_r2 = move_robot(char, NUM, robot1_pos, G_num, "1->2")
        print("1->2", movements_r2)
        
        all_r2_seqs = []
        for movmts_r2 in movements_r2:
            aggregated_mvmt_r2 = []
            best_r3_at_r2_level = []

            for char2 in movmts_r2:
                movements_r3 = move_robot(char2, DIR, robot2_pos, G_dir, "2->3")
                print("\t2->3", movements_r3)

                all_r3_seqs = []
                for movmts_r3 in movements_r3:
                    aggregated_mvmt_r3 = []
                    
                    for char3 in movmts_r3:
                        movements_you = move_robot(char3, DIR, robot3_pos, G_dir, "3->U")
                        print("\t\t3->U", movements_you)
                        aggregated_mvmt_r3 += movements_you[0]

                        robot3_pos = DIR[char3]
                    
                    all_r3_seqs.append(aggregated_mvmt_r3)   
                    print("\t\t", "-" * 50)
                
                r3_idx, smallest_r3_seq = sorted(enumerate(all_r3_seqs), key=lambda x: len(x[1]))[0]
                
                best_r3_at_r2_level += smallest_r3_seq
                print("\t\tX", smallest_r3_seq)
                
                aggregated_mvmt_r2 += movements_r3[r3_idx]
                       
                robot2_pos = DIR[char2]
                
            all_r2_seqs.append((aggregated_mvmt_r2, best_r3_at_r2_level))
            print("\tA", all_r2_seqs)
            
            print("\t", "-" * 50)
        smallest_r2_seq = sorted(all_r2_seqs, key=lambda x: len(x[1]))[0]
        print("*", smallest_r2_seq[1])
        final_seq += smallest_r2_seq[1]

        robot1_pos = NUM[char]

    return final_seq

    
def complexity(value):
    sequence = find_shortest_sequence1(value)
    print(value, len(sequence), sequence)
    return int(value[:-1]) * len(sequence)

In [ ]:
total = 0
for line in data.splitlines():
    total += complexity(line)
    
if DEBUG:
    print(total)
else:
    puzzle.answer_a = total

### Part 2

In [ ]:
# OMG, this one was heavy !!!
# Took me a long time to figure out the best path algorithm, first I tried all combinations but it does not fit
# Quite rapidly I understood that we needed to avoid spliting the same movements (but I deleted this code out of anger... RIP)
# Then it came naturally that caching would help recompute the same movements again and again
# Plus I grouped symbols from an A to the next since that helped me get rid of storing the positions of the 25 robots
# However, I sticked to BFS for a very long time and it was too big to have depth 22 in memory for a single digit
# Later I tried removing some caching but not all and that did nothing different
# FINALLY I moved to DFS and noticed that storing length was enough !!!


@functools.lru_cache(maxsize=10000)
def find_movements_for_next_robot(source_char, text, is_num):
    
    total_movements = []
    
    for char in text:
        if is_num:
            paths = nx.shortest_path(G_num, NUM[source_char], NUM[char])
        else:
            paths = nx.shortest_path(G_dir, DIR[source_char], DIR[char])

        all_movements = []
        
        for path in paths:
            movements = []

            for before, after in zip(path[:-1], path[1:]):
                dr, dc = (after[0] - before[0], after[1] - before[1])

                if (dr, dc) == (1, 0):
                    movements.append("v")
                elif (dr, dc) == (-1, 0):
                    movements.append("^")
                elif (dr, dc) == (0, 1):
                    movements.append(">")
                elif (dr, dc) == (0, -1):
                    movements.append("<")
                else:
                    raise ValueError((dr, dc))
                    
            all_movements.append(movements + ["A"])
        total_movements.append(all_movements)
        source_char = char

    return [list(chain(*p)) for p in product(*total_movements)]

In [ ]:
@functools.lru_cache(maxsize=10000)
def find_movements_for_next_robot(source_char, text, is_num):
    
    total_movements = []
    
    for char in text:
        if is_num:
            paths = nx.all_shortest_paths(G_num, NUM[source_char], NUM[char])
        else:
            paths = nx.all_shortest_paths(G_dir, DIR[source_char], DIR[char])

        all_movements = []
        
        for path in paths:
            movements = []

            for before, after in zip(path[:-1], path[1:]):
                dr, dc = (after[0] - before[0], after[1] - before[1])

                if (dr, dc) == (1, 0):
                    movements.append("v")
                elif (dr, dc) == (-1, 0):
                    movements.append("^")
                elif (dr, dc) == (0, 1):
                    movements.append(">")
                elif (dr, dc) == (0, -1):
                    movements.append("<")
                else:
                    raise ValueError((dr, dc))
                    
            all_movements.append(movements + ["A"])
        total_movements.append(all_movements)
        source_char = char

    return [list(chain(*p)) for p in product(*total_movements)]

In [ ]:
def compute_next_movements(robots_pos, value, i):
    
    movements_next = []
    
    for char in value:
        # print(char)
        if i == 0:
            movements_next += find_movements_for_next_robot(G_num, robots_pos[i], NUM[char])
            robots_pos[i] = NUM[char]
        else:
            movements_next += find_movements_for_next_robot(G_dir, robots_pos[i], DIR[char])
            robots_pos[i] = DIR[char]

    return robots_pos, "".join(movements_next)

In [ ]:
def complexity(value):

    nb_robots = 3
    sequence = value
    robots_pos = [NUM["A"]] + [DIR["A"]] * nb_robots

    for i in range(nb_robots):
        print(sequence)
        robots_pos, sequence = compute_next_movements(robots_pos, sequence, i)
    
    print(value, len(sequence), sequence)
    return int(value[:-1]) * len(sequence)

In [ ]:
def compute_complexity(length, line):
    return int(line[:-1]) * length

In [ ]:
total = 0
for line in data.splitlines():
    num_robot_pos = "A"
    length = 0
    
    for c in line:
        print(c)
        paths = find_movements_for_next_robot(num_robot_pos, c, is_num=True)
        num_robot_pos = c
        
        for i in range(25):
            print("\t", i, len(paths))
            # print(i, paths)
            # print([len(p) for p in paths])
            # print()
            next_paths = []
            for path in paths:
                path_pieces = []
                pieces = "".join(path).split("A")
                for piece in pieces[:-1]:
                    # print(piece, "A")
                    path_pieces.append(find_movements_for_next_robot("A", f"{piece}A", is_num=False))
                    # print(find_movements_for_next_robot("A", f"{piece}A", is_num=False))
                # print(path_pieces)
                # print([list(chain(*p)) for p in product(*path_pieces)])
                # print()
                next_paths += [list(chain(*p)) for p in product(*path_pieces)]

            min_length = sorted([len(p) for p in next_paths])[0]
            paths = {"".join(p) for p in next_paths if len(p) == min_length}
            paths = [list(p) for p in paths]
            # print([len(p) for p in next_paths])
            
        # print(i+1, paths)
        length += sorted([len(p) for p in paths])[0]

    total += compute_complexity(length, line)
    print()

if DEBUG:
    print(total)
else:
    puzzle.answer_b = total

In [ ]:
# New attempt with the path algorithm but still BFS

@functools.lru_cache(maxsize=50)
def find_movements_for_next_robot(source_char, char, is_num):
    
    if is_num:
        path = nx.shortest_path(G_num, NUM[source_char], NUM[char])
    else:
        path = nx.shortest_path(G_dir, DIR[source_char], DIR[char])

    movements = []

    for before, after in zip(path[:-1], path[1:]):
        dr, dc = (after[0] - before[0], after[1] - before[1])

        if (dr, dc) == (1, 0):
            movements.append("v")
        elif (dr, dc) == (-1, 0):
            movements.append("^")
        elif (dr, dc) == (0, 1):
            movements.append(">")
        elif (dr, dc) == (0, -1):
            movements.append("<")
        else:
            raise ValueError((dr, dc))

    c = Counter(movements)
    
    # "<" before "^" before "v" before ">" except missing corners !
    if (is_num and (c["<"] == 2 or (c["<"] == 1 and source_char == "0"))) or (not is_num and (c["<"] == 2 or (c["<"] == 1 and source_char == "^"))):
        # do < at the end because of missing corner
            return "^" * c["^"] + c["v"] * "v" + "<" * c["<"] + "A"
            
    if is_num and (c["v"] == 3 or (c["v"] == 2 and source_char == "4") or (c["v"] == 1 and source_char == "1")):
        # do v at the end because of missing corner
        return "<" * c["<"] + c[">"] * ">" + c["v"] * "v" + "A"
            
    if not is_num and c["^"] == 1 and source_char == "<":
        # do ^ at the end because of missing corner
        return "<" * c["<"] + c[">"] * ">" + c["^"] * "^" + "A"
             
    return "<" * c["<"] + "^" * c["^"] + c["v"] * "v" + c[">"] * ">" + "A"

In [ ]:
def get_next_word(word):
    total_next_path = ""
    dir_robot_pos = "A"
    
    for c in word:
        total_next_path += find_movements_for_next_robot(dir_robot_pos, c, is_num=False)
        dir_robot_pos = c
    
    return total_next_path

In [ ]:
def get_next_path(path):
    words = [f"{p}A" for p in path[:-1].split("A")]
    
    total_next_path = ""
    
    for word in words:
        total_next_path += get_next_word(word)
    
    return total_next_path

In [ ]:
total = 0
for line in data.splitlines():
    num_robot_pos = "A"
    length = 0
    
    for c in line:
        print(c)
        path = find_movements_for_next_robot(num_robot_pos, c, is_num=True)
        num_robot_pos = c
        
        for i in range(25):
            print("\t", i)
            path = get_next_path(path)

        length += len(path)
    total += compute_complexity(length, line)
    # print(len(path))

if DEBUG:
    print(total)
else:
    puzzle.answer_b = total

In [ ]:
## DFS with len only !

@functools.lru_cache(maxsize=5000)
def find_movements_for_next_robot(source_char, char, is_num):
    
    if is_num:
        path = nx.shortest_path(G_num, NUM[source_char], NUM[char])
    else:
        path = nx.shortest_path(G_dir, DIR[source_char], DIR[char])

    movements = []

    for before, after in zip(path[:-1], path[1:]):
        dr, dc = (after[0] - before[0], after[1] - before[1])

        if (dr, dc) == (1, 0):
            movements.append("v")
        elif (dr, dc) == (-1, 0):
            movements.append("^")
        elif (dr, dc) == (0, 1):
            movements.append(">")
        elif (dr, dc) == (0, -1):
            movements.append("<")
        else:
            raise ValueError((dr, dc))

    c = Counter(movements)
    
    # "<" before "^" before "v" before ">" except missing corners !
    if (is_num and source_char in ["0", "A"] and char in ["1", "4", "7"]) or (not is_num and source_char in ["^", "A"] and char == "<"):
        # do < at the end because of missing corner
            return "^" * c["^"] + c["v"] * "v" + "<" * c["<"]
            
    if is_num and source_char in ["1", "4", "7"] and char in ["0", "A"]:
        # do v at the end because of missing corner
        return "<" * c["<"] + c[">"] * ">" + c["v"] * "v"
            
    if not is_num and source_char == "<" and char in ["^", "A"]:
        # do ^ at the end because of missing corner
        return "<" * c["<"] + c[">"] * ">" + c["^"] * "^"
             
    return "<" * c["<"] + "^" * c["^"] + c["v"] * "v" + c[">"] * ">"


@functools.lru_cache(maxsize=5000)
def computation(source, target, depth):
    
    path = find_movements_for_next_robot(source, target, depth == 0)
    
    if depth == 25:
        return len(path) + 1
    
    return sum([computation(s, t, depth + 1) for s, t in zip("A" + path, path + "A")])


def compute_complexity(length, line):
    return int(line[:-1]) * length

In [ ]:
total = 0
for line in data.splitlines():
    num_robot_pos = "A"
    length = 0
    
    for c in line:
        calc = computation(num_robot_pos, c, 0)
        print(c, calc)
        length += calc
        num_robot_pos = c

    complexity = compute_complexity(length, line)
    total += complexity
    print(complexity)

if DEBUG:
    print(total)
else:
    puzzle.answer_b = total

## Day 20
https://adventofcode.com/2024/day/20

In [ ]:
puzzle = Puzzle(year=2024, day=20)
input_data = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""

DEBUG = False
data = input_data if DEBUG else puzzle.input_data

### Part 1

In [ ]:
G = nx.Graph()

init_grid = defaultdict(lambda: defaultdict(str))
init_pos = None
exit_pos = None
SYMBOLES = [0, 1, 2, 3]

for row_idx, line in enumerate(data.splitlines()):
    for col_idx, val in enumerate(line):        
        if val in ["S", "E"]:
            
            if val == "S":
                init_pos = (row_idx, col_idx)
            else:
                exit_pos = (row_idx, col_idx)
            val = "."
                
        if val == ".":
            init_grid[row_idx][col_idx] = "."
            
            G.add_node((row_idx, col_idx))
            
            for dr, dc in [(-1, 0), (0, -1)]:
                if init_grid[row_idx + dr][col_idx + dc] == ".":
                    G.add_edge((row_idx, col_idx), (row_idx + dr, col_idx + dc), weight=1)

In [ ]:
shortest = nx.shortest_path(G, init_pos, exit_pos, "weight")

distance_to = {init_pos: {}, exit_pos: {}}
N = len(shortest) - 1

DELTA = 2
LIMIT = 1 if DEBUG else 100

for idx, node in enumerate(shortest):
    distance_to[init_pos][node] = idx
    distance_to[exit_pos][node] = N - idx
    

c = Counter()
for node in shortest:
    for dr in range(-DELTA, DELTA+1):
        for dc in range(-DELTA+abs(dr), DELTA-abs(dr)+1):
            if init_grid[node[0] + dr][node[1] + dc] == ".":
                gain = N - distance_to[init_pos][node] - distance_to[exit_pos][(node[0] + dr, node[1] + dc)] - abs(dr) - abs(dc)
                # print(node, (node[0] + dr, node[1] + dc), gain)
                if gain >= LIMIT:
                    c[gain] += 1
    
if DEBUG:
    print(sum(c.values()))
else:
    puzzle.answer_a = sum(c.values())

In [ ]:
## Too slow !

shortest = nx.shortest_path_length(G, init_pos, exit_pos, "weight")
malus = 10 ** (1 + math.ceil(math.log10(shortest)))

grid = copy.deepcopy(init_grid)

for row_idx, line in grid.items():
    for col_idx, val in line.items():        
                
        if val == ".":
            
            for dr, dc in [(-1, 0), (0, -1)]:
                if init_grid[row_idx + dr][col_idx + dc] == "" and init_grid[row_idx + 2 * dr][col_idx + 2 * dc] == ".":
                    G.add_edge((row_idx, col_idx), (row_idx + 2*dr, col_idx + 2*dc), weight=malus)
                    
paths_gains = Counter()

for path in nx.shortest_simple_paths(G, init_pos, exit_pos, "weight"):
    N = nx.path_weight(G, path, "weight")
    
    if N > malus + shortest - 99:
        break
    
    if N > shortest:
        print(N)
        paths_gains[shortest - N + malus] += 1

### Part 2

In [ ]:
shortest = nx.shortest_path(G, init_pos, exit_pos, "weight")

distance_to = {init_pos: {}, exit_pos: {}}
N = len(shortest) - 1

DELTA = 20
LIMIT = 1 if DEBUG else 100

for idx, node in enumerate(shortest):
    distance_to[init_pos][node] = idx
    distance_to[exit_pos][node] = N - idx
    

c = Counter()
for node in shortest:
    for dr in range(-DELTA, DELTA+1):
        for dc in range(-DELTA+abs(dr), DELTA-abs(dr)+1):
            if init_grid[node[0] + dr][node[1] + dc] == ".":
                gain = N - distance_to[init_pos][node] - distance_to[exit_pos][(node[0] + dr, node[1] + dc)] - abs(dr) - abs(dc)
                # print(node, (node[0] + dr, node[1] + dc), gain)
                if gain >= LIMIT:
                    c[gain] += 1
    
if DEBUG:
    print(sum(c.values()))
else:
    puzzle.answer_b = sum(c.values())

## Day 19

https://adventofcode.com/2024/day/19

In [ ]:
puzzle = Puzzle(year=2024, day=19)

input_data = """r, wr, b, g, bwu, rb, gb, br

brwrr
bggr
gbbr
rrbgbr
ubwu
bwurrg
brgr
bbrgwb"""
DEBUG = False
data = input_data if DEBUG else puzzle.input_data

In [ ]:
patterns, designs = data.split("\n\n")
patterns = patterns.split(", ")
designs = designs.splitlines()

max_pattern_size = max([len(p) for p in patterns])

### Part 1

In [ ]:
valid_designs = set()

for design in designs:
    
    possible_cuts = {design}
    done_with_design = False
    
    while possible_cuts:
        next_possible_cuts = set()
        for cut in possible_cuts:
            # print(cut, done_with_design)
            if done_with_design:
                break

            for substr in range(1, max_pattern_size + 1):
                # print(cut[:substr], end=": ")
                if cut[:substr] in patterns:
                    # print("YES")
                    if cut[substr:] == "":
                        valid_designs.add(design)
                        done_with_design = True
                    else:
                        next_possible_cuts.add(cut[substr:])
                # else:
                    # print("NO")

        possible_cuts = copy.deepcopy(next_possible_cuts)
        # print(possible_cuts)
if DEBUG:
    print(len(valid_designs))
else:
    puzzle.answer_a = len(valid_designs)

### Part 2

In [ ]:
@functools.lru_cache(maxsize=300000)
def all_patterns(design):
    
    if design == "":
        return 1
    
    possibles = []
    for pattern in patterns:
        if design.startswith(pattern):
            N = len(pattern)
            possibles.append(all_patterns(design[N:]))
    
    print(possibles)
    return sum(possibles)

In [ ]:
patterns

In [ ]:
total = 0
for design in designs:
    total += all_patterns(design)
    
if DEBUG:
    print(total)
else:
    puzzle.answer_b = total

In [ ]:
valid_designs = set()

for design in designs:
    
    possible_cuts = {(design, ())}  # (remaining, actual_split)
    done_with_design = False
    
    while possible_cuts:
        next_possible_cuts = set()
        for cut in possible_cuts:
            remaining, actual = cut
            
            # print(cut)

            for substr in range(1, max_pattern_size + 1):
                # print(remaining[:substr], end=": ")
                if remaining[:substr] in patterns:
                    # print("YES")
                    if remaining[substr:] == "":
                        valid_designs.add(tuple(list(actual) + [remaining]))
                    else:
                        next_possible_cuts.add((remaining[substr:], tuple(list(actual) + [remaining[:substr]])))
                # else:
                    # print("NO")

        possible_cuts = copy.deepcopy(next_possible_cuts)
        # print(possible_cuts)
if DEBUG:
    print(len(valid_designs))
else:
    puzzle.answer_b = len(valid_designs)

In [ ]:
puzzle.answer_b = len(valid_designs)

## Day 18
https://adventofcode.com/2024/day/18

In [ ]:
puzzle = Puzzle(year=2024, day=18)
# X: col_idx, Y: row_idx
input_data = """5,4  
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0"""
DEBUG = False
data = input_data if DEBUG else puzzle.input_data

### Part 1

In [ ]:
exit = (70, 70)
position = (0, 0)
grid = defaultdict(str)
for r,c in product(range(71), repeat=2):
    grid[r, c] = "."
    
for line in data.splitlines()[:1024]:
    col_idx , row_idx = [int(x) for x in line.split(",")]
    grid[row_idx, col_idx] = "#"

In [ ]:
distance = defaultdict(lambda: -1)
distance[exit] = 0

queue = [(70, 70)]

while queue:
    row, col = queue.pop(0)    
    
    for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        next_pos = (row + dr, col + dc)
        if grid[next_pos] == ".":
            
            if distance[next_pos] == -1:
                distance[next_pos] = distance[row, col] + 1
                queue.append(next_pos)
                
            elif distance[next_pos] > distance[row, col] + 1:
                distance[next_pos] = distance[row, col] + 1
                
puzzle.answer_a = distance[0, 0]

### Part 2

In [ ]:
G = nx.Graph()
size = 7 if DEBUG else 71

for x, y in product(range(size), repeat=2):
    G.add_node((x,y))
    
    for dx, dy in [(0, -1), (-1, 0)]:
        if 0 <= x+dx < size and 0 <= y+dy < size:
            G.add_edge((x, y), (x+dx, y+dy))
            
for line in data.splitlines():
    x, y = [int(x) for x in line.split(",")]
    
    G.remove_node((x, y))
    
    if (size-1, size-1) not in nx.node_connected_component(G, (0,0)):
        if DEBUG:
            print(f"{x},{y}")
        else:
            puzzle.answer_b = f"{x},{y}"
        break

## Day 17
https://adventofcode.com/2024/day/17

In [ ]:
puzzle = Puzzle(year=2024, day=17)
input_data = """Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0"""

# for part B
input_data = """Register A: 2024
Register B: 0
Register C: 0

Program: 3,6,7,0,5,7,3,1,4"""
DEBUG = False
data = input_data if DEBUG else puzzle.input_data

In [ ]:
for idx, line in enumerate(data.splitlines()):
    if idx == 0:
        a = int(line[12:])
    elif idx == 1:
        b = int(line[12:])
    elif idx == 2:
        c = int(line[12:])
    elif idx == 4:
        init_program = [int(p) for p in line[9:].split(",")]

### Part 1

In [ ]:
def operations(index, registry):
    opcode = init_program[index]
    operand = init_program[index + 1]
    
    if opcode == 0:
        registry[4] //= 2 ** registry[operand]
        
    elif opcode == 1:
        registry[5] ^= operand
        
    elif opcode == 2:
        registry[5] = registry[operand] % 8
        
    elif opcode == 3:
        if registry[4]:
            return operand, registry, ""
    
    elif opcode == 4:
        registry[5] ^= registry[6]
        
    elif opcode == 5:
        return index + 2, registry, str(registry[operand] % 8)
        
    elif opcode == 6:
        registry[5] = registry[4] // (2 ** registry[operand])
        
    elif opcode == 7:
        registry[6] = registry[4] // (2 ** registry[operand])
        
    return index + 2, registry, ""

In [ ]:
index = 0
registry = [0, 1, 2, 3, a, b, c]
outputs = []

try:
    while True:
        index, registry, output = operations(index, registry)

        if output:
            outputs.append(output)
except IndexError as e:
    if DEBUG:
        print(",".join(outputs))
    else:
        puzzle.answer_a = ",".join(outputs)

### Part 2

In [ ]:
## Brute force, lol no ! (solution is in 2^48)
a_reg = 0

while True:
    index = 0
    registry = [0, 1, 2, 3, a_reg, b, c]
    outputs = []
    
    try:
        while True:
            index, registry, output = operations(index, registry)
    
            if output:
                if program[len(outputs)] == int(output):
                    outputs.append(int(output))
                else:
                    break
        
    except IndexError as e:
        if outputs == program:
            if DEBUG:
                print(a_reg)
            else:
                puzzle.answer_b = a_reg
            break
    finally:
        a_reg += 1

In [ ]:
def constrained_set(constraint):
    counter = Counter(constraint)
    nb_x = counter["x"]
    
    for possibility in product([0, 1], repeat=nb_x):
        total = 0
        idx = 0
        for c in constraint:
            if c == "x":
                total += possibility[idx]
                idx += 1
            else:
                total += int(c)
            total *= 2
            
        yield total // 2

In [ ]:
# reversed engineer program and reverse computed the binary strings then output the smallest valid candidate

program = copy.deepcopy(init_program)
print(program)

possibilities = [["xxx"]]  # start by saying it can be any string of size 3

while program:
    v = program.pop(0)
    # print("V=", v, ": ", possibilities)    
    
    new_possibilities = []
    for poss in possibilities:
    
        constraint = poss[-1]
        this_constraint, next_constraint = constraint[:3], constraint[3:]
        
        # print("P", poss, this_constraint, next_constraint)
        
        for a in constrained_set(this_constraint[::-1]):            
            b = a ^ 1
            output = '{0:03b}'.format(a ^ 4 ^ v)
            a_bin = '{0:03b}'.format(a)
            new_constraint = "x" * max(0, b-3) + output[::-1]
            
            # print(a, a ^ 4 ^ v, a_bin, new_constraint, b)
            
            if b < 3:
                rev_a_bin = a_bin[::-1]
                if rev_a_bin[b:] != new_constraint[:3-b]:
                    # print("DISRESPECT")
                    continue
                    
                new_constraint = new_constraint[3-b:]
                            
            if new_constraint == next_constraint == "":
                combined_constraint = "xxx"
            else:
                combined_constraint = ""

                for c1, c2 in zip_longest(new_constraint, next_constraint):
                    if c1 is None:
                        combined_constraint += c2
                    elif c2 is None:
                        combined_constraint += c1
                    elif c1 == "x":
                        combined_constraint += c2
                    elif c2 == "x":
                        combined_constraint += c1
                    elif c1 == c2:
                        combined_constraint += c1
                    else:
                        combined_constraint = ""
                        break
                        
            if combined_constraint:
                combined_constraint += "x" * max(0, 3 - len(combined_constraint))
                # print(combined_constraint)
                new_possibilities.append(poss[:-1] + [a_bin, combined_constraint])
            # else:
                # print("COMBINED DISRESPECT")

                
            
    possibilities = new_possibilities

In [ ]:
results = set()

for poss in new_possibilities:
    poss_str = "".join(poss[::-1])
    poss_nox = poss_str.replace("x", "0")
    poss_striped = poss_nox.lstrip("0")

    if len(poss_striped) > 3 * (len(init_program) - 1):
        results.add(int(poss_striped, 2))
        
puzzle.answer_b = sorted(results)[0]

## Day 16
https://adventofcode.com/2024/day/16

In [ ]:
puzzle = Puzzle(year=2024, day=16)
input_data = """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""

input_data = """#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################"""
DEBUG = False
data = input_data if DEBUG else puzzle.input_data

### Part 1

In [ ]:
G = nx.Graph()

init_grid = defaultdict(lambda: defaultdict(str))
init_pos = None
exit_pos = None
SYMBOLES = [0, 1, 2, 3]

for row_idx, line in enumerate(data.splitlines()):
    for col_idx, val in enumerate(line):        
        if val in ["S", "E"]:
            
            if val == "S":
                init_pos = (row_idx, col_idx)
            else:
                exit_pos = (row_idx, col_idx)
            val = "."
                
        if val == ".":
            init_grid[row_idx, col_idx] = "."
            
            for symb in SYMBOLES:
                G.add_node((row_idx, col_idx, symb))

            for s1, s2 in combinations(SYMBOLES, r=2):
                weight = 2000 if (s1 - s2) % 4 == 2 else 1000
                G.add_edge((row_idx, col_idx, s1), (row_idx, col_idx, s2), weight=weight)
            
            
            for symb, (dr, dc) in enumerate([(-1, 0), (0, 1), (1, 0), (0, -1)]):
                if init_grid[row_idx + dr, col_idx + dc] == ".":
                    G.add_edge((row_idx, col_idx, symb), (row_idx + dr, col_idx + dc, symb), weight=1)
                

In [ ]:
distances = {
    orient: nx.shortest_path_length(G, (init_pos[0], init_pos[1], 1), (exit_pos[0], exit_pos[1], orient), "weight")
    for orient in range(4)
}

puzzle.answer_a = min(distances.values())

### Part 2

In [ ]:
best_orient = sorted(distances.items(), key=lambda i: i[1])[0][0]

good_seats = set()

for path in nx.all_shortest_paths(G, (init_pos[0], init_pos[1], 1), (exit_pos[0], exit_pos[1], best_orient), "weight"):
    for node in path:
        good_seats.add((node[0], node[1]))

puzzle.answer_b = len(good_seats)

## Day 15
https://adventofcode.com/2024/day/15

In [ ]:
puzzle = Puzzle(year=2024, day=15)
input_data = """########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<"""

input_data = """#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^"""

input_data = """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

DEBUG = False
data = input_data if DEBUG else puzzle.input_data

In [ ]:
def print_grid(grid, robot):
    grid[robot[0]][robot[1]] = "@"
    for row in grid.values():
        print("".join(row.values()))
    grid[robot[0]][robot[1]] = "."

In [ ]:
def compute(grid):

    total = 0

    for row_idx, row in grid.items():
        for col_idx, val in row.items():
            if val in ["O", "["]:
                total += row_idx * 100 + col_idx
                
    return total

In [ ]:
DELTA = {
    "<" : (0, -1),
    ">" : (0, 1),
    "^" : (-1, 0),
    "v" : (1, 0),
}

### Part 1

In [ ]:
init_grid = defaultdict(lambda: defaultdict(str))
init_robot = None

grid_str, movements_str = data.split("\n\n")
for row_idx, line in enumerate(grid_str.splitlines()):
    for col_idx, val in enumerate(line):        
        if val == "@":
            init_robot = (row_idx, col_idx)
            val = "."
            
        init_grid[row_idx][col_idx] = val

movements = "".join(list(movements_str.splitlines()))

In [ ]:
def move_box(grid, box_pos, move):
    
    ri, ci = box_pos
    
    nb_boxes = 1
    
    while True:
        next_pos = (ri + nb_boxes * DELTA[move][0], ci + nb_boxes * DELTA[move][1])
        next_item = grid[next_pos[0]][next_pos[1]]
        
        if next_item == ".":
            grid[ri][ci] = "."
            for bi in range(nb_boxes):
                pos = (ri + (bi + 1) * DELTA[move][0], ci + (bi + 1) * DELTA[move][1])
                # print(pos)
                grid[pos[0]][pos[1]] = "O"
                # print_grid(grid)
            return True, grid  # move and grid changed
            
        elif next_item == "O":
            nb_boxes += 1
        
        elif next_item == "#":
            return False, grid  # did not move, no grid change
            
        else:
            raise ValueError(next_item)

In [ ]:
grid = copy.deepcopy(init_grid)
robot = init_robot

for move in movements:
    # print_grid(grid, robot)
    # print()
    # print("M", move)
    new_robot = (robot[0] + DELTA[move][0], robot[1] + DELTA[move][1])
    # print(new_robot)
    
    if grid[new_robot[0]][new_robot[1]] == ".":
        robot = new_robot
        
    elif grid[new_robot[0]][new_robot[1]] == "#":
        continue
        
    elif grid[new_robot[0]][new_robot[1]] == "O":
        moved, grid = move_box(grid, new_robot, move)
        if moved:
            # print(moved)
            robot = new_robot
        
    else:
        raise ValueError(new_robot, grid[new_robot[0]][new_robot[1]])

In [ ]:
if DEBUG:
    print(compute(grid))
else:
    puzzle.answer_a = compute(grid)

### Part 2

In [ ]:
init_grid = defaultdict(lambda: defaultdict(str))
init_robot = None

grid_str, movements_str = data.split("\n\n")
for row_idx, line in enumerate(grid_str.splitlines()):
    for col_idx, val in enumerate(line):        
        if val == "@":
            init_robot = (row_idx, 2 * col_idx)
            val = "."
        
        if val == "O":
            init_grid[row_idx][2 * col_idx] = "["
            init_grid[row_idx][2 * col_idx + 1] = "]"
        else:
            init_grid[row_idx][2 * col_idx] = val
            init_grid[row_idx][2 * col_idx + 1] = val

movements = "".join(list(movements_str.splitlines()))

In [ ]:
print_grid(init_grid, init_robot)

In [ ]:
def move_box2(grid, box_pos, move):
    
    ri, ci = box_pos
    init_box = grid[ri][ci]
    
    if move in ["<", ">"]:
        nb_boxes = 1

        while True:
            next_pos = (ri, ci + 2 * nb_boxes * DELTA[move][1])
            next_item = grid[next_pos[0]][next_pos[1]]
        
            if next_item == ".":
                grid[ri][ci] = "."
                for bi in range(nb_boxes):
                    pos = (ri, ci + (2 * bi + 1) * DELTA[move][1])
                    # print(pos)
                    grid[pos[0]][pos[1]] = init_box
                    grid[pos[0]][pos[1] + DELTA[move][1]] = "]" if init_box == "[" else "["
                    # print_grid(grid)

                return True, grid  # move and grid changed

            elif next_item in ["[", "]"]:
                nb_boxes += 1

            elif next_item == "#":
                return False, grid  # did not move, no grid change
            
            else:
                raise ValueError(next_item)
        
    else:
        cols_to_check = defaultdict(set)
        cols_to_check[ci] = {(ri, init_box)}
        cols_to_check[ci + (1 if init_box == "[" else -1)] = {(ri, "]" if init_box == "[" else "[")}
        
        while True:
            # print(cols_to_check)
            all_solved = True
            
            for col_idx, col in copy.deepcopy(cols_to_check).items():
                sorted_col = sorted(col, key=lambda x: x[0], reverse=move == "^")
                
                next_ri = sorted_col[-1][0] + DELTA[move][0]
                next_item = grid[next_ri][col_idx]
                # print(next_ri, next_item)
                
                if sorted_col[-1][1] == ".":
                    continue
                    # cols_to_check[col_idx].add((next_ri, next_item))

                elif sorted_col[-1][1] in ["[", "]"]:
                    all_solved = False

                    if next_item in [".", "[", "]"]:
                        cols_to_check[col_idx].add((next_ri, next_item))
                        
                        if next_item == "[":
                            cols_to_check[col_idx + 1].add((next_ri, "]"))
                        elif next_item == "]":
                            cols_to_check[col_idx - 1].add((next_ri, "["))
                    else:
                        return False, grid

                else:
                    # continue
                    raise ValueError(col)

            if all_solved:
                for col_idx, col in cols_to_check.items():
                    sorted_col = sorted(col, key=lambda x: x[0], reverse=move == "v")
                    
                    # print(col_idx, sorted_col)

                    for ri, val in sorted_col[1:]:
                        if val != ".":
                            grid[ri][col_idx] = "."  # will most likely be remodified by next line on next ri
                            grid[ri + (1 if move == "v" else -1)][col_idx] = val
                    grid[ri][col_idx] = "."

                return True, grid            

In [ ]:
grid = copy.deepcopy(init_grid)
robot = init_robot

for mi, move in enumerate(movements):
    
    # print_grid(grid, robot)
    # print()
    # print("M", mi, move)
    new_robot = (robot[0] + DELTA[move][0], robot[1] + DELTA[move][1])
    # print(new_robot)
    
    
    # At some point it was not working because I was modifying the number of boxes, breaking helped me fix it several times
    tot = 0
    for row in grid.values():
        tot += sum([int(r == "[") for r in row.values()])    
    if tot != 624:
        print(tot)
        break
    
    if grid[new_robot[0]][new_robot[1]] == ".":
        robot = new_robot
        
    elif grid[new_robot[0]][new_robot[1]] == "#":
        continue
        
    elif grid[new_robot[0]][new_robot[1]] in ["[", "]"]:
        moved, grid = move_box2(grid, new_robot, move)
        if moved:
            robot = new_robot
        
    else:
        raise ValueError(new_robot, grid[new_robot[0]][new_robot[1]])

In [ ]:
print_grid(grid, robot)

In [ ]:
if DEBUG:
    print(compute(grid))
else:
    puzzle.answer_b = compute(grid)

## Day 14
https://adventofcode.com/2024/day/14

In [ ]:
puzzle = Puzzle(year=2024, day=14)
input_data = """p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3"""
DEBUG = False
data = input_data if DEBUG else puzzle.input_data
size = (11, 7) if DEBUG else (101, 103)  # size is X x Y, display is Y x X


### Part 1

In [ ]:
positions_after_100 = defaultdict(lambda: defaultdict(int))
quadrants = defaultdict(lambda: defaultdict(int))
runs = 100

for robot in data.splitlines():
    match = re.match(r"p=(\d+),(\d+) v=(-*\d+),(-*\d+)", robot)
    x, y, vx, vy = [int(x) for x in match.groups()]
    # print(x, y, end=" -> ")
    
    x_100 = (x + runs * vx) % size[0]
    y_100 = (y + runs * vy) % size[1]
    
    positions_after_100[x_100][y_100] += 1
    
    # print(x_100, y_100, end=" ¬ ")
    if x_100 == ((size[0] - 1) // 2) or y_100 == ((size[1] - 1) // 2):
        # print()
        continue

    x_quad = int(x_100 > ((size[0] - 1) / 2))
    y_quad = int(y_100 > ((size[1] - 1) / 2))
    # print(x_quad, y_quad)
    quadrants[x_quad][y_quad] += 1
    
print(([list(v.values()) for v in quadrants.values()]))
puzzle.answer_a = 132*119*121*119

### Part 2

In [ ]:
grid = defaultdict(lambda: defaultdict(list))  # grid indexing is Y x X

for robot in data.splitlines():
    match = re.match(r"p=(\d+),(\d+) v=(-*\d+),(-*\d+)", robot)
    x, y, vx, vy = [int(x) for x in match.groups()]
    grid[y][x].append((vy, vx))

In [ ]:
def run(grid):
    new_grid = defaultdict(lambda: defaultdict(list))
    
    for row_idx, row in grid.items():
        for col_idx, elems in row.items():
            for elem in elems:
                vy, vx = elem
                new_grid[(row_idx + vy) % size[1]][(col_idx + vx) % size[0]].append(elem)
            
    return new_grid

In [ ]:
def display_tree(grid):
    for row_idx in range(size[1]):
        for col_idx in range(size[0]):
            print(len(grid[row_idx][col_idx]) or " ", end="")
        print()

In [ ]:
def count_consecutives(grid):
    count = 0
    
    for row_idx, row in copy.deepcopy(grid).items():
        for col_idx, elems in row.items():
            if grid[row_idx + 1][col_idx] or grid[row_idx][col_idx + 1]:
                count += 1
                
    return count            

In [ ]:
runs = defaultdict(list)
nb_run = 0

while True:
    if nb_run % 100 == 0:
        clear_output(wait=True)
        display(nb_run)
    
    runs[count_consecutives(grid)].append(nb_run)
    grid = run(grid)
    nb_run += 1
    
    
    if nb_run == 10001:
        break

In [ ]:
sorted(runs.keys(), reverse=True)[:10]

In [ ]:
runs[352]

In [ ]:
grid7502 = defaultdict(lambda: defaultdict(list))  # grid indexing is Y x X

for robot in data.splitlines():
    match = re.match(r"p=(\d+),(\d+) v=(-*\d+),(-*\d+)", robot)
    x, y, vx, vy = [int(x) for x in match.groups()]
    grid7502[(y + 7502 * vy) % size[1]][(x + 7502 * vx) % size[0]].append((vy, vx))

In [ ]:
display_tree(grid7502)

## Day 13

_TBD_

https://adventofcode.com/2024/day/13

In [ ]:
puzzle = Puzzle(year=2024, day=13)
DEBUG = False
input_data = """Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279"""
data = input_data if DEBUG else puzzle.input_data

### Part 1

In [ ]:
cost_a = 3
cost_b = 1
tokens = 0

for game in data.split("\n\n"):
    btn_a, btn_b, prize = game.splitlines()
    btn_a_re = re.match(r"Button \w: X\+(\d+), Y\+(\d+)", btn_a)
    btn_b_re = re.match(r"Button \w: X\+(\d+), Y\+(\d+)", btn_b)
    prize_re = re.match(r"Prize: X=(\d+), Y=(\d+)", prize)
    x_a, y_a = [int(elem) for elem in btn_a_re.groups()]
    x_b, y_b = [int(elem) for elem in btn_b_re.groups()]
    x_prize, y_prize = [int(elem) for elem in prize_re.groups()]
    
    matrice = [[x_a, x_b, x_prize], [y_a, y_b, y_prize]]
    # print(matrice)
    
    div_b = x_b / y_b
    matrice[0] = [x_a - div_b * y_a, x_b - div_b * y_b, x_prize - div_b * y_prize]
    
    # print(matrice)
    
    matrice[0] = [1, 0, matrice[0][2] / matrice[0][0]]
    matrice[1] = [0, matrice[1][1], matrice[1][2] - matrice[1][0] * matrice[0][2]]
    matrice[1] = [0, 1, matrice[1][2] / matrice[1][1]]
    
    if matrice[1] == [0,0,0]:
        print("X", matrice)
        
    # print(matrice)
        
    if round(matrice[0][2]) * x_a + round(matrice[1][2]) * x_b == x_prize and round(matrice[0][2]) * y_a + round(matrice[1][2]) * y_b == y_prize:
        # print(cost_a * round(matrice[0][2]) + cost_b * round(matrice[1][2]))
        tokens += cost_a * round(matrice[0][2]) + cost_b * round(matrice[1][2])

if DEBUG:
    print(tokens)
else:
    puzzle.answer_a = tokens

### Part 2

In [ ]:
delta = 10000000000000

cost_a = 3
cost_b = 1
tokens = 0

for game in data.split("\n\n"):
    btn_a, btn_b, prize = game.splitlines()
    btn_a_re = re.match(r"Button \w: X\+(\d+), Y\+(\d+)", btn_a)
    btn_b_re = re.match(r"Button \w: X\+(\d+), Y\+(\d+)", btn_b)
    prize_re = re.match(r"Prize: X=(\d+), Y=(\d+)", prize)
    x_a, y_a = [int(elem) for elem in btn_a_re.groups()]
    x_b, y_b = [int(elem) for elem in btn_b_re.groups()]
    x_prize, y_prize = [delta + int(elem) for elem in prize_re.groups()]
    
    matrice = [[x_a, x_b, x_prize], [y_a, y_b, y_prize]]
    # print(matrice)
    
    div_b = x_b / y_b
    matrice[0] = [x_a - div_b * y_a, x_b - div_b * y_b, x_prize - div_b * y_prize]
    
    # print(matrice)
    
    matrice[0] = [1, 0, matrice[0][2] / matrice[0][0]]
    matrice[1] = [0, matrice[1][1], matrice[1][2] - matrice[1][0] * matrice[0][2]]
    matrice[1] = [0, 1, matrice[1][2] / matrice[1][1]]
    
    if matrice[1] == [0,0,0]:
        print("X", matrice)
        
    # print(matrice)
        
    if round(matrice[0][2]) * x_a + round(matrice[1][2]) * x_b == x_prize and round(matrice[0][2]) * y_a + round(matrice[1][2]) * y_b == y_prize:
        # print(cost_a * round(matrice[0][2]) + cost_b * round(matrice[1][2]))
        tokens += cost_a * round(matrice[0][2]) + cost_b * round(matrice[1][2])

if DEBUG:
    print(tokens)
else:
    puzzle.answer_b = tokens

## Day 12

_TBD_

https://adventofcode.com/2024/day/12

In [ ]:
puzzle = Puzzle(year=2024, day=12)
DEBUG = False
input_data = """EEEEE
EXXXX
EEEEE
EXXXX
EEEEE"""

In [ ]:
grid = create_grid(input_data if DEBUG else puzzle.input_data)
grid_cp = copy.deepcopy(grid)

areas = defaultdict(list)

for row_idx, row in grid.items():
    for col_idx, val in row.items():
        to_print = False
        
        if to_print:
            print("X", row_idx, col_idx, val)
        
        areas_idx = set()
        
        for i, area in enumerate(areas[val]):
            if to_print:
                print("A", i, area)
                
            if (row_idx - 1, col_idx) in area:
                areas_idx.add(i)
            
            if (row_idx, col_idx -1) in area:
                areas_idx.add(i)
                
        areas_idx = list(areas_idx)
        
        if to_print:
            print("M", areas_idx)
        
        if len(areas_idx) == 0:
            areas[val].append([(row_idx, col_idx)])
        elif len(areas_idx) == 1:
            areas[val][areas_idx[0]].append((row_idx, col_idx))
        elif len(areas_idx) == 2:
            areas_idx.sort()
            list1 = areas[val][areas_idx[0]]
            list2 = areas[val][areas_idx[1]]
            areas[val].append(list1 + list2 + [(row_idx, col_idx)])
            del areas[val][areas_idx[0]]
            del areas[val][areas_idx[1] - 1]
            
        if to_print:
            print("N", areas[val])

### Part 1

In [ ]:
perims = defaultdict(lambda: defaultdict(int))

for row_idx, row in grid_cp.items():
    for col_idx, val in row.items():
        
        areas_idx = -1
        for i, area in enumerate(areas[val]):
            if (row_idx, col_idx) in area:
                areas_idx = i
                break
                
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                if dr != dc and dr * dc == 0:
                    
                    # each neighbor of another kind requires a fence
                    if grid[row_idx + dr][col_idx + dc] != val:
                        perims[val][areas_idx] += 1

In [ ]:
total = 0
for val, v_areas in areas.items():
    for i, area in enumerate(v_areas):
        # print(val, len(area), "*", perims[val][i])
        total += len(area) * perims[val][i]
        
if DEBUG:
    print(total)
else:
    puzzle.answer_a = total

### Part 2

In [ ]:
perims_set = defaultdict(lambda: defaultdict(list))
KIND = {
    (-1, 0): "D",
    (1, 0): "U",
    (0, -1): "R",
    (0, 1): "L",
}

for row_idx, row in grid_cp.items():
    for col_idx, val in row.items():
        
        areas_idx = -1
        for i, area in enumerate(areas[val]):
            if (row_idx, col_idx) in area:
                areas_idx = i
                break
                
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                if dr != dc and dr * dc == 0:
                    
                    # each neighbor of another kind requires a fence
                    if grid[row_idx + dr][col_idx + dc] != val:
                        perims_set[val][areas_idx].append((row_idx + dr, col_idx + dc, KIND[(dr, dc)]))

In [ ]:
total = 0
for val, v_areas in areas.items():
    for i, area in enumerate(v_areas):        
        
        cells = perims_set[val][i]
        split_cells = []
        
        for cell in cells:
            row_idx, col_idx, kind = cell
            
            for i, spc in enumerate(split_cells):
                if (row_idx - 1, col_idx, kind) in spc or (row_idx, col_idx -1, kind) in spc:
                    split_cells[i].append(cell)
                    break
            else:
                split_cells.append([cell])
        
        if DEBUG:
            print(val, len(area), "*", len(split_cells))
            print(sorted(perims_set[val][i]))
            print()
            for sc in split_cells:
                print(sc)
            print()
        
        total += len(area) * len(split_cells)
        
if DEBUG:
    print(total)
else:
    puzzle.answer_b = total

In [ ]:
for row in grid.values():
    print("".join(row.values()))

## Day 11

_TBD_

https://adventofcode.com/2024/day/11

In [ ]:
puzzle = Puzzle(year=2024, day=11)
input_data = """125 17"""
DEBUG = False
data = input_data if DEBUG else puzzle.input_data

### Part 1

In [ ]:
def blink_at_stone(stone):
    
    if stone == 0:
        return [1]
    
    stone_str = str(stone)
    n = len(stone_str)
    
    if n % 2 == 0:
        return [int(stone_str[:n//2]), int(stone_str[n//2:])]
    else:
        return [stone * 2024]

In [ ]:
# Brute force
stones = [int(d) for d in data.split(" ")]

for i in range(25):
    new_stones = []
    while stones:
        stone = stones.pop(0)
        new_stones += blink_at_stone(stone)
        
    stones = new_stones

In [ ]:
# Smarter algo

stones = Counter([int(d) for d in data.split(" ")])
blinking_book = defaultdict(list)

for i in range(25):
    new_stones = Counter()
    
    for stone, count in stones.items():
        if stone not in blinking_book:
            blinking_book[stone] = blink_at_stone(stone)
            
        for new_val in blinking_book[stone]:
            new_stones[new_val] += count
        
    stones = new_stones
    
puzzle.answer_a = sum(new_stones.values())

### Part 2

In [ ]:
for i in range(50):
    new_stones = Counter()
    
    for stone, count in stones.items():
        if stone not in blinking_book:
            blinking_book[stone] = blink_at_stone(stone)
            
        for new_val in blinking_book[stone]:
            new_stones[new_val] += count
        
    stones = new_stones
    
puzzle.answer_b = sum(new_stones.values())

## Day 10

_TBD_

https://adventofcode.com/2024/day/10

In [ ]:
puzzle = Puzzle(year=2024, day=10)
DEBUG = False
input_data = """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""

data = input_data if DEBUG else puzzle.input_data

### Part 1

In [ ]:
grid = defaultdict(lambda: defaultdict(int))
queue = []

for row_idx, line in enumerate(data.splitlines()):
    for col_idx, val in enumerate(line):
        grid[row_idx][col_idx] = int(val)
        if val == "0":
            queue.append([0, (row_idx, col_idx)])

In [ ]:
trails = set()
nb_trails = 0

while queue:
    # print("Q", queue)

    cur_val, *trail = queue.pop()
    cur_row, cur_col = trail[-1]
    # print("C", cur_val, cur_row, cur_col)
    for dr, dc in product(range(-1, 2), repeat=2):
        if dr != 0 and dc != 0:
            continue
            
        if grid[cur_row + dr][cur_col + dc] == cur_val + 1:
            # print(dr, dc)
            if cur_val == 8:
                trails.add((trail[0][0], trail[0][1], cur_row + dr, cur_col + dc))
                nb_trails += 1
            else:
                queue.append([cur_val + 1, *trail, (cur_row + dr, cur_col + dc)])
                
if DEBUG:
    print(len(trails))
else:
    puzzle.answer_a = len(trails)
    puzzle.answer_b = nb_trails

### Part 2

In [ ]:
puzzle.answer_a = len(trails)

## Day 9

_TBD_

https://adventofcode.com/2024/day/9

In [ ]:
puzzle = Puzzle(year=2024, day=9)
DEBUG = False
input_data = """2333133121414131402"""

In [ ]:
data = input_data if DEBUG else puzzle.input_data
fulls = [(e, int(data[i])) for e,i in enumerate(range(0, len(data),2))]
emptys = [int(data[i]) for i in range(1, len(data),2)]

### Part 1

In [ ]:
disk = []
try:
    while True:
        val, count = fulls.pop(0)
        # print("0", val, count)
        disk += count * [val]
        # print(disk)

        empty_count = emptys.pop(0)
        # print("1", empty_count)

        while empty_count > 0:
            val, count = fulls.pop()
            # print("2", val, count)

            if empty_count < count:
                fulls.insert(len(fulls), (val, count - empty_count))

            # in all cases
            disk += min(count, empty_count) * [val]
            empty_count -= count
            # print("3", empty_count)
            
except IndexError:
    puzzle.answer_a = sum([e * d for e, d in enumerate(disk)])

### Part 2

In [ ]:
disk = [[(".", e), f] for e, f in zip(emptys, fulls[1:])]
disk = [fulls[0]] + list(chain(*disk))
if DEBUG:
    print(disk)

In [ ]:
def compute(outputs):
    total = 0
    idx = 0
    for v, c in outputs:
        for j in range(c):
            if v != ".":
                total += idx * v
            idx += 1

    return total

In [ ]:
queue = deque(disk)
outputs = []

try:
    while True:
        val, count = queue.pop()

        if val != "." and val % 1000 == 0:
            print("P", val)
        # print("Q", queue)

        pasted = False

        if val != ".":
            copy_q = copy.deepcopy(queue)

            for idx, elem in enumerate(copy_q):
                val_elem, count_elem = elem

                if val_elem == "." and count_elem >= count:
                    # print("E", elem)

                    queue.remove(elem)
                    if count_elem != count:
                        queue.insert(idx, (".", count_elem - count))
                    queue.insert(idx, (val,count))

                    pasted = True
                    break

        if not pasted:
            outputs = [(val, count)] + outputs
            # print("O", outputs)
        else:
            last_val, last_count = queue.pop()
            if last_val == ".":
                queue.append((".", last_count + count))
            else:
                queue.append((last_val, last_count))
                queue.append((".", count))
                
except IndexError:
    if DEBUG:
        print(outputs)
        print(compute(outputs))
    else:
        puzzle.answer_b = compute(outputs)

## Day 8

_vectors_

https://adventofcode.com/2024/day/8

In [ ]:
puzzle = Puzzle(year=2024, day=8)
DEBUG = False
input_data = """............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............"""

### Part 1

In [ ]:
antenas = defaultdict(list)
for row_idx, line in enumerate((input_data if DEBUG else puzzle.input_data).splitlines()):
    for col_idx, val in enumerate(line):
        if val != ".":
            antenas[val].append((row_idx, col_idx))
size = (row_idx + 1, col_idx + 1)

In [ ]:
antinodes = set()
for ant_positions in antenas.values():
    for elem1, elem2 in permutations(ant_positions, 2):
        if 0 <= 2 * elem1[0] - elem2[0] < size[0] and 0 <= 2 * elem1[1] - elem2[1] < size[1]:
            antinodes.add((2 * elem1[0] - elem2[0], 2 * elem1[1] - elem2[1]))
        
if DEBUG:
    print(antinodes)
else:
    puzzle.answer_a = len(antinodes)

### Part2

In [ ]:
antinodes = set()
for ant_positions in antenas.values():
    for elem1, elem2 in permutations(ant_positions, 2):
        coef = 0
        while 0 <= (1 + coef) * elem1[0] - coef * elem2[0] < size[0] and 0 <= (1+coef) * elem1[1] - coef * elem2[1] < size[1]:
            antinodes.add(((1 + coef) * elem1[0] - coef * elem2[0], (1 + coef) * elem1[1] - coef * elem2[1]))
            coef += 1
        
if DEBUG:
    print(antinodes)
else:
    puzzle.answer_b = len(antinodes)

## Day 7

_recursion_

https://adventofcode.com/2024/day/7

In [ ]:
puzzle = Puzzle(year=2024, day=7)
DEBUG = False
input_data = """7290: 6 8 6 15"""

In [ ]:
data = input_data.splitlines() if DEBUG else puzzle.input_data.splitlines()

### Part 1

In [ ]:
def can_compute(total, values):
    if len(values) == 1:
        return total == values[0]
    
    last = values.pop()
    return can_compute(total / last, copy.deepcopy(values)) or can_compute(total - last, copy.deepcopy(values))

In [ ]:
total = 0
for line in data:
    tot, values = line.split(": ")
    if can_compute(int(tot), [int(v) for v in values.strip().split(" ")]):
        total += int(tot)
        
if DEBUG:
    print(total)
else:
    puzzle.answer_a = total

### Part 2

In [ ]:
def can_compute(total, values):
    if int(total) != total or total < 0:
        return False
    total = int(total)

    if len(values) == 1:
        return total == values[0]
    
    last = values.pop()
    operators_computable = can_compute(total / last, copy.deepcopy(values)) or can_compute(total - last, copy.deepcopy(values))
    
    last = str(last)

    if operators_computable:
        return True
    elif str(total)[-len(last):] == last and len(str(total)) > len(last):
        return can_compute(int(str(total)[:-len(last)]), copy.deepcopy(values))
    else:
        return False

In [ ]:
total = 0
for line in data:
    tot, values = line.split(": ")
    if can_compute(int(tot), [int(v) for v in values.strip().split(" ")]):
        total += int(tot)
        
if DEBUG:
    print(total)
else:
    puzzle.answer_b = total

## Day 6

_grid search, brute force_

https://adventofcode.com/2024/day/6

In [ ]:
puzzle = Puzzle(year=2024, day=6)
DEBUG = False
input_data = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""
grid = create_grid(input_data if DEBUG else puzzle.input_data)

In [ ]:
def find_in_grid(search, grid):
    for r_idx, row in grid.items():
        for c_idx, elem in row.items():
            if elem == search:
                return (r_idx, c_idx)
            
    raise ValueError

### Part 1

In [ ]:
def move_on_grid(cur_pos, symbol, grid):
    if symbol == "^":
        delta = [-1, 0]
        next_symbol = ">"
    elif symbol == "<":
        delta = [0, -1]
        next_symbol = "^"
    elif symbol == ">":
        delta = [0, 1]
        next_symbol = "v"
    elif symbol == "v":
        delta = [1, 0]
        next_symbol = "<"
    else:
        raise ValueError
        
    check_pos = grid[cur_pos[0] + delta[0]][cur_pos[1] + delta[1]]

    if check_pos == "":
        return None, None
    elif check_pos == "#":
        return (cur_pos[0], cur_pos[1], next_symbol)
    elif check_pos in [".", "^"]:
        return (cur_pos[0] + delta[0], cur_pos[1] + delta[1], symbol)
    else:
        raise IndexError

In [ ]:
def find_all_positions(grid, init_pos, init_symb):
    positions = defaultdict(set)
    
    pos = init_pos
    symb = init_symb

    while True:
        positions[pos].add(symb)
        *next_pos, next_symb = move_on_grid(pos, symb, grid)
        # print(*next_pos, next_symb)

        if next_pos[0] is None:
            exit = True
            return exit, positions
        elif next_symb in positions.get((next_pos[0], next_pos[1]), []):
            exit = False  # hence loop
            return exit, positions
        else:
            pos = (next_pos[0], next_pos[1])
            symb = next_symb    

In [ ]:
pos = find_in_grid("^", grid)
symb = grid[pos[0]][pos[1]]
exit, positions = find_all_positions(grid, pos, symb)
puzzle.answer_a = len(positions)

### Part 2

In [ ]:
obstacles = set()
positions = defaultdict(set)
init_pos = find_in_grid("^", grid)
init_symb = grid[init_pos[0]][init_pos[1]]

pos = init_pos
symb = init_symb

while True:
    positions[pos].add(symb)
    *next_pos, next_symb = move_on_grid(pos, symb, grid)
    # print(*next_pos, next_symb)
    
    if next_pos[0] is None:
        break
        
    elif symb == next_symb:  # can try to add an obstacle and see if guard loops
        new_grid = copy.deepcopy(grid)
        new_grid[next_pos[0]][next_pos[1]] = "#"
        exit, _ = find_all_positions(new_grid, init_pos, init_symb)
        if not exit:
            obstacles.add((next_pos[0], next_pos[1]))
        
    # then move on
    pos = (next_pos[0], next_pos[1])
    symb = next_symb

if DEBUG:
    print(len(obstacles), obstacles)
else:
    puzzle.answer_b = len(obstacles)

In [ ]:
init_pos = find_in_grid("^", grid)
init_symb = grid[init_pos[0]][init_pos[1]]

end_status = []

# For some unknown reason some obstacles are wrongly put :shrug:
for obstacle in obstacles:
    new_grid = copy.deepcopy(grid)
    new_grid[obstacle[0]][obstacle[1]] = "#"
    
    exit, _ = find_all_positions(new_grid, init_pos, init_symb)
    end_status.append(exit)

In [ ]:
puzzle.answer_b = len(obstacles) - sum(end_status)

## Day 5

_recursion_

https://adventofcode.com/2024/day/5

In [ ]:
puzzle = Puzzle(year=2024, day=5)
input_data = """"""

In [ ]:
rulz, prints = puzzle.input_data.split("\n\n")
rules = defaultdict(set) # k not before any v
for rul in rulz.splitlines():
    first, sec = rul.split("|")
    rules[first].add(sec)

### Part 1

In [ ]:
def correct_printing_list(pages_before: list[str], page_to_print: str) -> bool:
    """Define whether the page_to_print can come at the place in the printing"""
    
    if not pages_before:
        return True

    if rules[page_to_print].intersection(set(pages_before)):
        return False
    
    page_to_print = pages_before.pop()
    return correct_printing_list(pages_before, page_to_print)

In [ ]:
total = 0
for printing in prints.splitlines():
    print_list = printing.split(",")
    middle_idx = (len(print_list) - 1) // 2
    
    if correct_printing_list(print_list[:-1], print_list[-1]):
        total += int(print_list[middle_idx])
        
puzzle.answer_a = total

### Part 2

In [ ]:
def order_printing(pages: list[str], sub_rules: dict[str]) -> list[str]:
    """Order pages to print following given set of rules"""
    
    if len(pages) == 1:
        return pages
    
    new_sub_rules = {p: sub_rules[p].intersection(pages) for p in pages}
    
    for page, const in new_sub_rules.items():
        if not const:
            pages.remove(page)
            return [page] + order_printing(pages, new_sub_rules)

    raise ValueError  # at least one empty const should exist

In [ ]:
total = 0
for printing in prints.splitlines():
    print_list = printing.split(",")
    middle_idx = (len(print_list) - 1) // 2
    
    if not correct_printing_list(print_list[:-1], print_list[-1]):
        ordered_print_list = order_printing(print_list, rules)
        total += int(ordered_print_list[middle_idx])
        
puzzle.answer_b = total

## Day 4

_Grid search_

https://adventofcode.com/2024/day/4

In [ ]:
puzzle = Puzzle(year=2024, day=4)
input_data = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""

In [ ]:
DEBUG = False
iterator = input_data.splitlines() if DEBUG else puzzle.input_data.splitlines()

grid = []
for row in iterator:
    grid.append(list(row))
    
sizes = (len(grid), len(row))

### Part 1

In [ ]:
def count_combinations(grid:list[list[str]], r_idx: int, c_idx:int) -> int:
    """Count number of X-MAS starting with a X at the given position"""
    
    if grid[r_idx][c_idx] != "X":
        return 0
    
    attempts = []
    for dr in range(-1, 2):
        for dc in range(-1, 2):
            
            if DEBUG:
                print("start", dr, dc)
                
            attempt = True
            for (coef, letter) in [(1, "M"), (2, "A"), (3, "S")]:
                try:
                    if r_idx + coef * dr < 0 or c_idx + coef * dc < 0:
                        attempt = False
                    if grid[r_idx + coef * dr][c_idx + coef * dc] != letter:
                        attempt = False
                except IndexError:
                    attempt = False
                    
                if DEBUG:
                    print(coef, attempt)
                    
            attempts.append(attempt)
    return sum(attempts)

In [ ]:
nb_combinations = []

for r_idx in range(0, sizes[0]):
    row_combinations = []
    
    for c_idx in range(0, sizes[1]):
        row_combinations.append(count_combinations(grid, r_idx, c_idx))
    nb_combinations.append(row_combinations)
    
if DEBUG:
    print(sum([sum(row) for row in nb_combinations]))
else:
    puzzle.answer_a = sum([sum(row) for row in nb_combinations])

### Part 2

In [ ]:
def is_a_crossing(grid:list[list[str]], r_idx: int, c_idx:int) -> bool:
    """Define whether the given position is the center of an X-MAS crossing"""
    
    if grid[r_idx][c_idx] != "A":
        return False
    
    attempts = []
    for dr in range(-1, 2):
        for dc in range(-1, 2):
            
            if DEBUG:
                print("start", dr, dc)
                
            attempt = True
            for (coef, letter) in [(-1, "M"), (1, "S")]:
                try:
                    if r_idx + coef * dr < 0 or c_idx + coef * dc < 0:
                        attempt = False
                    if grid[r_idx + coef * dr][c_idx + coef * dc] != letter:
                        attempt = False
                except IndexError:
                    attempt = False
                    
                if DEBUG:
                    print(coef, attempt)
                    
            if attempt:
                attempts.append([dr, dc])
                
    if DEBUG:
        print("attempts", attempts)
        
    return ([-1, 1] in attempts or [1, -1] in attempts) and ([1, 1] in attempts or [-1, -1] in attempts)

In [ ]:
nb_combinations = []

for r_idx in range(0, sizes[0]):
    row_combinations = []
    
    for c_idx in range(0, sizes[1]):
        row_combinations.append(is_a_crossing(grid, r_idx, c_idx))
    nb_combinations.append(row_combinations)
    
if DEBUG:
    print(sum([sum(row) for row in nb_combinations]))
else:
    puzzle.answer_b = sum([sum(row) for row in nb_combinations])

## Day 3

_regular expressions_

https://adventofcode.com/2024/day/3

In [ ]:
puzzle = Puzzle(year=2024, day=3)

### Part 1

In [ ]:
total = 0
for x in re.findall(r'mul\(\d{1,3},\d{1,3}\)', puzzle.input_data):
    a, b = x[4:-1].split(",")
    total += int(a) * int(b)
puzzle.answer_a = total

### Part 2

In [ ]:
total = 0
do = True
for x in re.findall(r"mul\(\d{1,3},\d{1,3}\)|do\(\)|don\'t\(\)", puzzle.input_data):
    if x == "do()":
        do = True
    elif x == "don't()":
        do = False
    elif do:
        a, b = x[4:-1].split(",")
        total += int(a) * int(b)
puzzle.answer_b = total

## Day 2

_defaultdict, min, max_

https://adventofcode.com/2024/day/2

In [ ]:
puzzle = Puzzle(year=2024, day=2)

In [ ]:
reports = puzzle.input_data.splitlines()

### Part 1

In [ ]:
def is_safe(report: list[int]) -> bool:
    """Ensure if report is safe"""
    
    differences: list[int] = [int(x[0]) - int(x[1]) for x in zip(report[1:], report[:-1])]
        
    return all([1 <= d <= 3 for d in differences]) or all([-3 <= d <= -1 for d in differences])

In [ ]:
puzzle.answer_a = len(list(filter(is_safe, reports)))

### Part 2

In [ ]:
safe_dampened = int(puzzle.answer_a)

for report in filter(lambda r: not is_safe(r), reports):
    for sub_report in combinations(report, len(report) - 1):
        if is_safe(sub_report):
            safe_dampened += 1
            break
            
puzzle.answer_b = safe_dampened

## Day 1

_sorting, counters_

https://adventofcode.com/2024/day/1

In [ ]:
puzzle = Puzzle(year=2024, day=1)

In [ ]:
left_list, right_list = [], []
for row in puzzle.input_data.splitlines():
    left, right = row.split()
    left_list.append(int(left))
    right_list.append(int(right))



### Part 1

In [ ]:
differences = 0

for sorted_left, sorted_right in zip(sorted(left_list), sorted(right_list)):
    differences += abs(sorted_left - sorted_right)
    
puzzle.answer_a = differences

### Part 2

In [ ]:
counter = Counter(right_list)

In [ ]:
similarity = 0

for elem in left_list:
    similarity += elem * counter.get(elem, 0)
    
puzzle.answer_b = similarity